In [16]:
# importing required libraries
from os import walk

import pathlib

import sys

!{sys.executable} -m pip install ffprobe-python
!{sys.executable} -m pip install ffmpeg-python

from ffprobe import FFProbe
import ffmpeg

In [21]:

class Report:
    def __init__(self, file_path):
        self.file_path = file_path
        self.report = ''
        self.Fixed_file_path = ''

    def isOk(self):
        return 'Video file is OK.' in self.report
    
    def add_report(self, message):
        self.report += message + '\n'

In [22]:
# Verifies that the input file complies with all criteria

# Runs FFProbe on a file and checks if it meets the criteria.
def check_file(file_path):
    report = Report(file_path)
    ext = os.path.splitext(file_path)[1].lower()
    if ext != '.mp4':
        report.add_report(f'-- Wrong video container format. Expected mp4, found {ext}')
        return report

    Meta_Data = ffmpeg.probe(file_path)
    for s in Meta_Data['streams']:
        if s['codec_type'] == 'video':
            video_codec = s['codec_name']
            width = s['width']
            height = s['height']
            fps = s['r_frame_rate']
            break
    
    if not video_codec:
        report.add_report('-- No video stream found.')
        return report
    
    if video_codec.lower() not in ('h264', 'hevc'):
        report.add_report(f'-- Wrong video codec. Expected h264 or hevc, found {video_codec}.')
    
    if int(fps.split('/')[0]) < 25:
        report.add_report(f'-- Wrong fps. Expected >= 25, found {fps}.')
        
    if width < 640 or height < 360:
        report.add_report(f'-- Wrong frame size. Expected >= 640x360, found {width}x{height}.')
    
    if report.isOk():
        report.add_report('Video file is OK.')
    
    return report

In [24]:
import os

# Uses ffmpeg to convert a file that doesn't meet the criteria.
def fix_file(report):
    # new file path is set
    ext = '.mp4'
    report.Fixed_file_path = f'{os.path.splitext(report.file_path)[0]}_formatOK{ext}'
    
    # extracting audio and video streams
    stream = ffmpeg.input(report.file_path)
    audio = stream.audio
    video = stream.video
    
    # setting fps
    video = video.filter('fps', fps=25, round='up') 
    
    # setting frame size and resolution
    video = video.filter('scale', width='640', height='360')
    
    # video and audio codec, video and audio bitrate are set
    output = ffmpeg.output(audio, video, report.Fixed_file_path, vcodec="libx264", acodec="aac", video_bitrate="3M", audio_bitrate="256K") 
    
    # saving the file
    output.overwrite_output().run()

# Prints the file report.

def print_report(report):
    if not report.messages:
        print(f'{report.file_name} is OK')
    else:
        print(f'{report.file_name} has issues:')
        for message in report.messages:
            print(message)

# File path
film_folder_path = "Exercise3_Films"
file_names = [f for f in os.listdir(film_folder_path) if os.path.isfile(os.path.join(film_folder_path, f))]

for f in file_names:
    # Delete any files produced by earlier iterations of the algorithm.
    if '_formatOK' in f:
        os.remove(os.path.join(film_folder_path, f))

    # Check the file.
    file_path = os.path.join(film_folder_path, f)
    report = check_file(file_path)

    # Print the results.
    print_report(report)

FileNotFoundError: [WinError 2] 系统找不到指定的文件。